# ParaView Volume Rendering on parallel nodes

In [ ]:
from paraview.simple import *
from paraview.modules.vtkRemotingCore import vtkProcessModule

Using a terminal on this node, we request a new allocation:

salloc -N 4 -n 4 -A csstaff -C gpu --reservation tutorial --time=00:20:00

salloc: Granted job allocation 30807331
salloc: Waiting for resource configuration
salloc: Nodes nid0[3508-3511] are ready for job

####################

We run pvserver on the newly allocated nodes

srun pvserver

The jupyter client must connect to the first MPI task on the new allocation

In [ ]:
#Connect("localhost")
Connect("nid03508")

In [ ]:
#gs = GetSettingsProxy("RenderViewSettings")
#gs.SetPropertyWithName("DisableIceT", 0)

rank = vtkProcessModule.GetProcessModule().GetPartitionId()
nbprocs = servermanager.ActiveConnection.GetNumberOfDataPartitions()
info = GetOpenGLInformation(location=servermanager.vtkSMSession.RENDER_SERVER)

In [ ]:
Version = str(GetParaViewVersion())
if rank == 0:
    print("ParaView Version ", Version)
    print("rank=", rank, "/", nbprocs)
    print("Vendor:   %s" % info.GetVendor())
    print("Version:  %s" % info.GetVersion())
    print("Renderer: %s" % info.GetRenderer())

In [ ]:
reader = ImageReader(FileNames=['/scratch/snx3000/jfavre/NVIDIA/50.density-log_1024x1024x2048_float.raw'])
reader.DataScalarType = 'float'
reader.DataByteOrder = 'LittleEndian'
reader.DataExtent = [0, 2047, 0, 1023, 0, 1023]

In [ ]:
renderView = GetActiveView()
rep = Show(reader, GetActiveView())
rep.Representation = 'Outline'
rep.ColorArrayName = ['POINTS', 'ImageFile']

ImageFileLUT = GetColorTransferFunction('ImageFile')
ImageFileLUT.RGBPoints = [1.206409215927124, 0.231373, 0.298039, 0.752941, 5.729784369468689, 0.865003, 0.865003, 0.865003, 10.253159523010254, 0.705882, 0.0156863,
 0.14902]
ImageFileLUT.ScalarRangeInitialized = 1.0

# get opacity transfer function/opacity map for 'MetaImage'
ImageFilePWF = GetOpacityTransferFunction('ImageFile')
ImageFilePWF.Points = [1.206409215927124, 0.0, 0.5, 0.0, 10.253159523010254, 1.0, 0.5, 0.0]
ImageFilePWF.ScalarRangeInitialized = 1


In [ ]:
from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(GetActiveView())
w = display(pvdisp)

In [ ]:
def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(pvdisp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

# save animage on the client-side
SaveImage("/users/jfavre/foo2.png")

In [ ]:
rep.Representation = 'Volume'
rep.LookupTable = ImageFileLUT
rep.OpacityArray = ['POINTS', 'ImageFile']
rep.OpacityTransferFunction = 'PiecewiseFunction'
rep.ScalarOpacityUnitDistance = 4
rep.ScalarOpacityFunction = ImageFilePWF

In [ ]:
ResetCamera()

We can cleanly close the connection with the following command

In [ ]:
Disconnect()